# Initialize

In [ ]:
!pip install p3_data openpyxl

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib
import re

In [ ]:
import p3_data
from p3_data import (glob_file_list , load_json_from_file, merge_dicts, plot_groups, 
    get_varying_column_names, filter_dataframe, take_varying_columns,
    load_json_records_as_dataframe, flatten_multiindex_columns)

In [ ]:
pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 50

# Load and Clean Results

In [ ]:
%%time
# Load result files from P3 Test Driver
src_files = []
src_files += ['../data/p3_test_driver/results/*.json.bz2']
raw_df = load_json_records_as_dataframe(src=src_files, ignore_error=True)

In [ ]:
raw_df.columns

In [ ]:
def parse_images_per_sec(output):
    """Search for last occurance of: total images/sec: 8169.88"""
    try:
        for m in re.finditer('^total images/sec: ([.0-9]+)$', output, flags=re.MULTILINE):
            pass
        return float(m.groups()[0])
    except:
        return np.nan

In [ ]:
data_dir_suffix_info_df = pd.DataFrame([
    {'data_dir_suffix': '', 'mean_bytes_per_image': 147531475882/1281167},
    {'data_dir_suffix': '-0.1x', 'mean_bytes_per_image': 147531475882/1281167},
    {'data_dir_suffix': '-7x', 'mean_bytes_per_image': 147531475882/1281167},
    {'data_dir_suffix': '-150x', 'mean_bytes_per_image': 147531475882/1281167},
    {'data_dir_suffix': '-png2', 'mean_bytes_per_image': 1750011440471/1281167},
]).set_index(['data_dir_suffix'])

In [ ]:
# Clean raw results
def clean_result(result):
    try:
        r = result.copy()
        r['utc_begin'] = pd.to_datetime(r['utc_begin'], utc=True)
        r['utc_end'] = pd.to_datetime(r['utc_end'], utc=True)
        r['num_gpus'] = r['np']
        r['images_per_sec'] = parse_images_per_sec(r['output'])
        r['images_per_sec_per_gpu'] = r['images_per_sec'] / r['num_gpus']
        r['image_format'] = 'JPEG'
        return pd.Series(r)
    except Exception as e:
        print('ERROR: %s: %s' % (r['test_uuid'], e))
        # raise e

In [ ]:
#r = clean_result(raw_df.iloc[-1])
#pd.DataFrame(r)

In [ ]:
clean_df = raw_df.apply(clean_result, axis=1)
clean_df = clean_df.set_index('test_uuid', drop=False)
clean_df = clean_df[clean_df.error==False]
clean_df = clean_df.sort_values(['utc_begin'])
clean_df['num_copies'] = clean_df['num_copies'].fillna(1.0)
clean_df.image_format = clean_df.image_format.fillna('JPEG')
clean_df = clean_df.join(data_dir_suffix_info_df, on=['data_dir_suffix'])
clean_df['MB_per_sec'] = clean_df['images_per_sec'] * clean_df['mean_bytes_per_image'] * 1e-6

In [ ]:
# Clean other raw results
def clean_other_result(result):
    try:
        r = result.copy()        
        r['images_per_sec_per_gpu'] = r['images_per_sec'] / r['np']
        r['NVIDIA_TENSORFLOW_VERSION'] = str(r['NVIDIA_TENSORFLOW_VERSION'])
        return pd.Series(r)
    except Exception as e:
        #print('ERROR: %s: %s' % (r['test_uuid'], e))
        raise e

In [ ]:
if False:
    raw_other_results_df = pd.read_csv('other_benchmark_results.csv')
    clean_other_results_df = raw_other_results_df.apply(clean_other_result, axis=1)
    combined_df = pd.concat([clean_df, clean_other_results_df], sort=False)
else:
    combined_df = clean_df

# Explore data

In [ ]:
# Show list of columns
list(clean_df.columns.values)

In [ ]:
# Define columns that identify test parameters
param_cols = [
 'NVIDIA_TENSORFLOW_VERSION',
 'TENSORFLOW_VERSION',
 'batch_group_size',
 'batch_size',
 'cached',
 'datasets_num_private_threads',
 'datasets_prefetch_buffer_size',
 'fp16',
 'use_tf_layers',
 'image_format',
 'model',
 'mpi',
 'np',
 'num_batches',
 'num_gpus',
 'num_hosts',
 'num_inter_threads',
 'num_intra_threads',
 'nvlink',
 'storage_type',
 'tensorflow_benchmark_git_hash',
 'test']

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
    'utc_begin',    
    'images_per_sec',
    'images_per_sec_per_gpu',
    'MB_per_sec',
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
# View most recent results
clean_df[cols].tail(3).T

In [ ]:
# Export to CSV
clean_df[cols].to_csv('results.csv')

In [ ]:
clean_df.groupby(['model']).size()

In [ ]:
clean_df.groupby(['storage_type']).size()

# Full Comparison

In [ ]:
# First level of filtering
filt_df = filter_dataframe(
    clean_df,
    batch_size=64,
    cached=False,
#     image_format='PNG',
    num_batches=(500,50000),
    num_copies=7,
)
len(filt_df)

In [ ]:
filt1_df = filter_dataframe(
    filt_df,
#     storage_type='isilon',
#     NVIDIA_TENSORFLOW_VERSION='19.09',
#      model='resnet50',
#     np=48,
#     num_batches=(500,50000),
#     cached=False,
#     num_copies=150,
#     image_format='JPEG',
)
take_varying_columns(filt1_df[cols].sort_values(['MB_per_sec'], ascending=False)).T

In [ ]:
df = filt1_df.fillna(0).groupby([
    #'NVIDIA_TENSORFLOW_VERSION',
    #'fp16',
    'model',
    'num_gpus',
#     'mpi',
#     'batch_size',
    'storage_type',
#      'cached',
#     'use_tf_layers',
#     'nvlink',
#     'datasets_num_private_threads',
#     'num_inter_threads',
#     'num_intra_threads',
]).agg({'images_per_sec_per_gpu': ['mean','count'],
        'images_per_sec': ['min','mean','max'],
        'MB_per_sec': ['mean']})
df

In [ ]:
df2 = df.unstack([
#     'num_gpus',
    #'datasets_num_private_threads',
#     'batch_size',
#     'NVIDIA_TENSORFLOW_VERSION',
#     'cached',
    'storage_type',
#     'use_tf_layers',
#     'nvlink',
])
df2

In [ ]:
df2[('MB_per_sec','mean')]

In [ ]:
df3 = df.unstack([
    'num_gpus',
    #'datasets_num_private_threads',
#     'batch_size',
#     'NVIDIA_TENSORFLOW_VERSION',
#     'cached',
    'storage_type',
#     'use_tf_layers',
#     'nvlink',
])
df3

In [ ]:
flatten_multiindex_columns(df3).to_excel('training_results.xlsx')

# ResNet50 Scaling

In [ ]:
filt_df = filter_dataframe(
    clean_df,
    batch_size=64,
#     cached=False,
    model='resnet50',
    num_batches=(500,50000),
    num_copies=[0.1,7],
)
len(filt_df)

In [ ]:
filt1_df = filter_dataframe(
    filt_df,
    num_gpus=(9,120),
)
len(filt1_df)

In [ ]:
tz = 'America/Los_Angeles'
filt1_df.utc_begin.min().astimezone(tz=tz), filt1_df.utc_end.max().astimezone(tz=tz)

In [ ]:
take_varying_columns(filt_df[cols].sort_values(['MB_per_sec'], ascending=False)).T

In [ ]:
df = filt_df.fillna(0).groupby([
    #'NVIDIA_TENSORFLOW_VERSION',
    #'fp16',
#     'model',
    'num_gpus',
#     'mpi',
#     'batch_size',
     'cached',
    'storage_type',
#     'use_tf_layers',
#     'nvlink',
#     'datasets_num_private_threads',
#     'num_inter_threads',
#     'num_intra_threads',
]).agg({'images_per_sec_per_gpu': ['mean','count'],
        'images_per_sec': ['min','mean','max'],
        'MB_per_sec': ['mean']})
df

In [ ]:
df2 = df.unstack([
#     'num_gpus',
    #'datasets_num_private_threads',
#     'batch_size',
#     'NVIDIA_TENSORFLOW_VERSION',
    'cached',
    'storage_type',
#     'use_tf_layers',
#     'nvlink',
])
df2

In [ ]:
flatten_multiindex_columns(df2).to_excel('resnet50_scaling.xlsx')

In [ ]:
ax = df2[('MB_per_sec','mean')].plot(style='-x', grid=True, figsize=(12,6))
ax.set_ylabel('Total MB/sec');

# GPU Metrics

In [ ]:
filt_df = filter_dataframe(
    clean_df,
    batch_size=64,
    cached=False,
    model='resnet50',
    num_batches=(500,50000),
    num_copies=7,
    num_gpus=1*4,
    storage_type='isilon',
)
len(filt_df)

In [ ]:
#take_varying_columns(filt_df[cols].sort_values(['MB_per_sec'], ascending=False)).T

In [ ]:
r = filt_df.iloc[-1]
nvidia_df = pd.read_csv(StringIO(r.metrics['nvidia_smi']['output']))
nvidia_df['timestamp_utc'] = pd.to_datetime(nvidia_df['timestamp'], utc=True)
nvidia_df['gpu_util_pct'] = nvidia_df[' utilization.gpu [%]'].str.replace(' %','').astype(np.float)
gpu0_df = nvidia_df[nvidia_df[' pci.bus_id']==' 00000000:00:04.0']
ax = gpu0_df.set_index('timestamp_utc')['gpu_util_pct'].plot(ylim=[0,100], figsize=(12,4))
ax.set_title('GPU Utilization, %s, %d GPUs' % (r.model, r.num_gpus));